In [3]:
from __future__ import absolute_import, division, print_function

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import linear_model

print(tf.__version__)


1.13.1


In [4]:
train = pd.read_csv("xaa.csv")
test = pd.read_csv("test.csv")

In [5]:
print(train.shape, test.shape)

(499999, 25) (13716, 26)


In [6]:
print(train.head(5))

   soldierId     shipId  attackId  assists  greekFireItems  healthLost  \
0        0.0       24.0       0.0      0.0             5.0      247.30   
1        1.0   440875.0       1.0      1.0             0.0       37.65   
2        2.0   878242.0       2.0      0.0             1.0       93.73   
3        3.0  1319841.0       3.0      0.0             0.0       95.88   
4        4.0  1757883.0       4.0      0.0             1.0        0.00   

   knockedOutSoldiers  throatSlits  healingPotionsUsed  killRank  ...  \
0                 2.0          0.0                 4.0      17.0  ...   
1                 1.0          1.0                 0.0      45.0  ...   
2                 1.0          0.0                 2.0      54.0  ...   
3                 0.0          0.0                 0.0      86.0  ...   
4                 0.0          0.0                 1.0      58.0  ...   

   numSaves  horseRideDistance  horseRideKills  swimmingDistance  \
0       1.0              591.3             0.0  

In [7]:
print(test.shape)

(13716, 26)


In [8]:
print(test.head(5))

   Unnamed: 0  index  soldierId     shipId  attackId  assists  greekFireItems  \
0           0      0    47734.0  1659463.0   47734.0      0.0             0.0   
1           1      1    47735.0  1659508.0   47735.0      0.0             1.0   
2           2      2    47736.0  1659555.0   47736.0      0.0             0.0   
3           3      3    47737.0  1659621.0   47737.0      0.0             0.0   
4           4      4    47738.0  1659675.0   47738.0      0.0             1.0   

   healthLost  knockedOutSoldiers  throatSlits  ...  numShips  numSaves  \
0       100.0                 1.0          0.0  ...      45.0       0.0   
1       400.0                 2.0          0.0  ...      47.0       1.0   
2         0.0                 0.0          0.0  ...      28.0       0.0   
3        68.6                 0.0          0.0  ...      92.0       0.0   
4       370.5                 3.0          0.0  ...      26.0       0.0   

   horseRideDistance  horseRideKills  swimmingDistance  friend

In [9]:
train.pop("shipId")
test.pop("shipId")
train.pop("soldierId")
test.pop("Unnamed: 0")
test.pop("index")
z = test.pop("soldierId")


In [10]:
train_labels = train.pop("bestSoldierPerc")

In [9]:
train_stats = train.describe()
train_stats = train_stats.transpose()
print(train_stats)

                        count          mean           std    min       25%  \
attackId             499999.0  23300.182682  13782.654589    0.0  11368.50   
assists              499999.0      0.326127      0.750084    0.0      0.00   
greekFireItems       499999.0      1.061976      1.638130    0.0      0.00   
healthLost           499999.0    154.894372    201.232546    0.0     18.15   
knockedOutSoldiers   499999.0      0.809920      1.420531    0.0      0.00   
throatSlits          499999.0      0.287293      0.704668    0.0      0.00   
healingPotionsUsed   499999.0      1.312619      2.495531    0.0      0.00   
killRank             499999.0     43.832834     27.588729    1.0     19.00   
killPoints           499999.0   1096.003176    134.507233  167.0   1000.00   
enemiesKilled        499999.0      1.128310      1.885243    0.0      0.00   
killingStreaks       499999.0     10.118118     29.226530    0.0      0.00   
farthermostKill      499999.0      8.540401     54.039077 -999.6

In [11]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = train
normed_test_data = test
print(normed_test_data.shape)

(13716, 22)


In [16]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64,  input_shape=[len(train.keys())]),
    layers.Dense(64),
    
    

    
    
    
    
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_absolute_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error'])
  return model

In [17]:
model = build_model()


In [18]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                1472      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 5,697
Trainable params: 5,697
Non-trainable params: 0
_________________________________________________________________


In [19]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
print(example_result)

[[ 0.10932225]
 [ 0.01986434]
 [-0.6101764 ]
 [ 0.57085896]
 [ 0.41320002]
 [ 0.01393642]
 [ 0.15233855]
 [ 0.21707721]
 [-0.2487609 ]
 [ 0.28500688]]


In [20]:
model = build_model()

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 20

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])





....................

In [21]:
test_predictions = model.predict(normed_test_data).flatten()


In [22]:
print(test.shape)

(13716, 22)


In [23]:
print(test_predictions)

[0.45546126 0.4768381  0.25434497 ... 0.2998839  0.19344595 0.8612559 ]


In [24]:
col_names = ['soldierId', 'bestSoldierPerc']
output = pd.DataFrame(columns = col_names)

In [25]:
output['soldierId'] = z
output['bestSoldierPerc'] = test_predictions

In [26]:
import numpy as np
output['soldierId'] = output['soldierId'].astype(np.int64)
output.set_index("soldierId", inplace=True)
print(output.head(5))

           bestSoldierPerc
soldierId                 
47734             0.455461
47735             0.476838
47736             0.254345
47737             0.329849
47738             0.111142


In [27]:
output.to_csv("output.csv")